**Imports**

In [ ]:
 import requests
import pandas as pd
import ipywidgets as widgets
from google.colab.widgets import Grid

**Create a data frame from the dataset**

In [ ]:
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3'
pd.set_option("display.max_columns", None)
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)

data_df = pd.DataFrame(data['result']['records'])

data_df.head()

,_id,mispar_rechev,tozeret_cd,sug_degem,tozeret_nm,degem_cd,degem_nm,ramat_gimur,ramat_eivzur_betihuty,kvutzat_zihum,shnat_yitzur,degem_manoa,mivchan_acharon_dt,tokef_dt,baalut,misgeret,tzeva_cd,tzeva_rechev,zmig_kidmi,zmig_ahori,sug_delek_nm,horaat_rishum,moed_aliya_lakvish,kinuy_mishari
0,1,7709728,683,P,סוזוקי-יפן,351,FHX51S,GL,NaN,NaN,2001,M13A,2024-07-11,2024-12-03,פרטי,JSAFHX51S10100474,25,כסף,165/70R14,165/70R14,בנזין,NaN,None,IGNIS
1,2,85729303,676,P,סקודה צ'כיה,1569,NX-NX34JD,STYLE,5.0,7.0,2024,DFY,2024-08-07,2025-08-06,פרטי,TMBAR8NXXRY178334,22,אפור כהה,225/45 R18,225/45 R18,בנזין,200950.0,2024-8,OCTAVIA
2,3,9801152,676,P,סקודה צ'כיה,302,5E3434,ELEGANCE,NaN,15.0,2014,CLH,2023-03-27,2024-03-10,פרטי,TMBAG7NE5E0147077,80,שנהב לבן,205/55 R16,205/55 R16,דיזל,140049.0,2014-3,OCTAVIA
3,4,53052702,676,P,סקודה צ'כיה,1065,NX33LD,DYNAMIC,2.0,7.0,2021,DLA,2024-03-07,2025-03-09,פרטי,TMBAP7NX8MY028697,29,כסף מטלי,205/55 R17,205/55 R17,בנזין,200950.0,2021-3,OCTAVIA
4,5,45282502,143,P,ב מ וו גרמניה,1296,71DY,BUSINESS,3.0,12.0,2021,B48B20A,2024-05-15,2025-01-19,פרטי,WBA71DY06MFK66386,15,אפור בהיר מטלי,225/50R17,225/50R17,בנזין,200998.0,2021-1,318I


**Create a dictionary of the unique manufacturers and the cars they make**

In [ ]:
maker_dict = {}
# For each car in the data frame
for index, car in data_df.iterrows():
  # If the current "totzeret" does not exist yet in the dictionary, add a new value list to that key
  if maker_dict.get(car["tozeret_nm"]) == None:
    maker_dict[car["tozeret_nm"]] = []
  # Add a new "kinuy_mishari" to the list of the current car manufacturer if it does not exist yet
  if car["kinuy_mishari"] not in maker_dict[car["tozeret_nm"]]:
    maker_dict[car["tozeret_nm"]].append(car["kinuy_mishari"])

# Example:
print("List of all models for maker Suzuki-Japan", maker_dict["סוזוקי-יפן"])

List of all models for maker Suzuki-Japan ['IGNIS', 'ליאנה', 'GRAND VITARA', 'SX4', 'SWIFT', 'LIANA', 'JIMNY', 'BALENO', 'SUPER BALENO', 'איגניס', 'סוזוקי יפן']


**Create the widgets and display the unique colors for each car**

In [ ]:
# Create a car manufacturer dropdown widget with all possible options
car_dropdown = widgets.Dropdown(
    options=list(maker_dict.keys()),
    description='Manufacturer:',
    style={'description_width': 'initial'}
)

# Create a car model dropdown widget with all possible options for the manufacturer "car_dropdown.value"
model_dropdown = widgets.Dropdown(
    options=maker_dict[car_dropdown.value],
    description='Model:',
    style={'description_width': 'initial'}
)

# Once the car manufacturer dropdown choice changes, change the model dropdown
def update_model_dropdown(change):
    # "change" contains the currently changed widget, column "new" is the new value name
    selected_maker = change['new']
    model_dropdown.options = maker_dict.get(selected_maker, [])  # Handle missing keys

# Updates the grid to display all of the possible "Ramot_gimur" for the selected model
def print_ramat_gimur(selected_model):
    # Create new list
    list_unique_ramat_gimur = []
    # For each car in the dataset, where "kinuy_mishari" equals to the chosen car model
    for index, row in data_df[data_df['kinuy_mishari'] == selected_model].iterrows():
        # if the current car's "ramat_gimur" does not exist yet, add it to the list of unique "ramot gimur"
        if row["ramat_gimur"] not in list_unique_ramat_gimur:
            list_unique_ramat_gimur.append(row["ramat_gimur"])

    # Display the list of ramot gimur for this model in index (1,1) of the grid
    with grid.output_to(1, 1):
      # Clear the current list of "ramot gimur"
      grid.clear_cell()

      ############### Cosmetic ####################
      # This for loop is only here to make the "ramat_gimur" output go from ["yellow", "blue", "red"] to yellow, blue, red in the grid.
      string_of_unique_ramat_gimur = ""
      for index, ramat_gimur in enumerate(list_unique_ramat_gimur):
        if index != len(list_unique_ramat_gimur) - 1:
          string_of_unique_ramat_gimur += ramat_gimur + ", "
        else:
          string_of_unique_ramat_gimur += ramat_gimur
      ############### Cosmetic ####################
      print(string_of_unique_ramat_gimur)

# Once the button is pressed, update the grid to show number of cars of the specific model and their "Ramat gimur"
def button_press(change):
    # "change" contains the currently changed widget, column "new" is the new value name
    selected_model = change['new']

    # Update the grid to display the number of cars
    with grid.output_to(0, 1):
      grid.clear_cell()
      print(len(data_df[data_df['kinuy_mishari'] == selected_model]))

    # Call function "print_ramat_gimur" to update the grid with the unique style a model can have
    print_ramat_gimur(selected_model)


# Connect car dropdown change to update function
car_dropdown.observe(update_model_dropdown, names='value')

# Update car dropdown options with maker names
car_dropdown.options = list(maker_dict.keys())

# A button which updates the data in the grid
update_button = widgets.Button(description='Update Records')
update_button.on_click(lambda b: button_press({'new': model_dropdown.value}))

# Display the widgets below
display(widgets.VBox([car_dropdown, model_dropdown, update_button]))
print("\n")

# Create a grid to display data
grid = Grid(2, 2, header_row=False, header_column=False)

with grid.output_to(0, 0):
  print("Total records:")

with grid.output_to(1, 0):
  print("Unique Ramat Gimur:")

,
,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Total records:


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PREMIUM, SPORT, INVITE, EXECUTEVE, EXECUTIVE, LIMITED, INSTYLE, GLX, EXECUTI PLUS, COMFORT, GL, EXCLUSIVE, SALON GLI, SPORT F


<IPython.core.display.Javascript object>